In [1]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=1000)
L=LaurentSeriesRing(QQ,'x', default_prec=1000)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=1000)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.

def uniteList(lst):
    ln=len(lst)
    if ln<2:return lst
    answer=[]
    for j in [0..ln-1]:
        lj=lst[j]
        if (lj in answer)==False:
            answer=answer+[lj]
    return answer

def delete_element(elt,ls):
    new=[]
    for j in ls:
        if j!=elt:new=new+[j]
    return new

def sortAndSmash(lst): #sorts and deletes repeats
    cropped=lst
    new=[]
    while len(cropped)>0:
        m=min(cropped)
        new=new+[m]
        cropped=delete_element(m,cropped)
    return(new)

def difference_table(lst):
    diffs=[]
    for k in [0..len(lst)-2]:
        diff=lst[k+1]-lst[k]
        diffs=diffs+[diff]
    return diffs

def evaluate(expression,incomingvalue):
    expr=str(expression)
    output=sage_eval(expr,locals={'x':incomingvalue})
    return output

P=Primes()

def prime(n): return P.unrank(n-1)

def unite(A,B):
    return set(A).union(B)

def factor_integer(n): #this is like "factor(integer)", but
    #the bases and exponents can be easily extracted.
    F=factor(n)
    return list(F)

def orderInteger(p,n):
    if n!=0:
        lst=n.digits(p)
        count=0
        k=0
        while lst[k]==0:
            count=count+1
            k=k+1
        return count
    if n==0:return(Infinity)

def order(p,fr):
    nn=numerator(QQ(fr))
    dn=denominator(QQ(fr))
    return orderInteger(p,nn)-orderInteger(p,dn)

def digitsum(p,n):return add(n.digits(p))

def digitsum(p,n):
    return sum(n.digits(p))

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])

def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer

def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))

def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))

def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)

def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))

def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)

def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)

def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # 
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.

def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)

def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]

def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)

def j(n,m):return L(xjNew(n,m)/x)

def xjLeoNew(n,m,multiplier):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(multiplier)**i
        return rt
    return xjNewStep(n-1,m)

def jLeo(n,m,multiplier):return L(xjLeoNew(n,m,multiplier)/x)

def JpowerConstantTerm(m,power):
    trun=J(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def jpowerConstantTerm(m,power):
    trun=j(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    
    return polynomialCoefficient(dg,polynomial)

def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent

def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)

def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())

def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())

def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr

def EulerMultiplication(listo):
    # Lemma 2.11 from my 1998 Experimental Mathematics paper.
    # Here the set A in the lemma is {1, 2, ...} = Z^+.
    p_A=[1] # so p_A[0]=1 as on top of p. 266, column 2.
    def ex(n):return listo[n-1] # ex is the exponent on the factor 1-x^n: -f(n)/n.
    def f(n):return -n*ex(n)
    answer=[1]
    def f_A(k):
        summ=0
        dk=divisors(k)
        for d in dk: #Here is where A = Z^+ is used to drop a constraint on k.
            summ=summ+f(d)
        return summ
    for n in [1..len(listo)-1]:
        sm=0
        for k in [1..n]:
            sm=sm+f_A(k)*p_A[n-k]
        p_A=p_A+[sm/n]
    return p_A

def listToPoly(listo):
    lenth=len(listo)
    answer=0
    for k in [1..lenth]:
        cf=listo[k-1]
        answer=answer+cf*x^(k-1)
    return answer
    
#Broadhurst code from #A005148:

#? {nt=100; a=[1]; b=[1]; d=1; e=0; g=0; print(1); for(n=2, nt, c=48*(a[n-1]+g)+128*(d-32*e); 
#e=d; d=c; i=(n-1)\2; g=12*if(n%2==0, a[n/2]^2)+24*sum(j=1, i, 
#a[j]*a[n-j]); h=12*if(n%2==0, b[n/2]^2)+24*sum(j=1, i, b[j]*b[n-j]); f=(c+5*h)/n^2-g; a=concat(a, f); 
#b=concat(b, n*f); print(f))}



def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

print("okay1")

okay1


In [8]:
for n in [1..5]:
    print((n,sigma(n,k=11)))

(1, 1)
(2, 2049)
(3, 177148)
(4, 4196353)
(5, 48828126)


In [2]:
import time


def p2RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 293
    ans=0
    for a in [1..n+1]:
        o2=order(2,a)
        purepart=2^o2
        oddpart=a/purepart
        d=sigma(oddpart,k=11)
        
        term=d*8^o2*x^a        
        ans=ans+term
    return D(ans)

def recipP2(n):
    ans=L(1/p2RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def p2RecipPowerConstantTerm(k):
    g=recipP2(k)^k
    g=g.O(k+1)
    #print("g:");print(g);print()
    #print("list:")
    gl=g.list()
    #print(gl);print()
    ln=len(gl)
    return gl[ln-2]
    

L=LaurentSeriesRing(QQ,'x', default_prec=1010)

tests=[]
start=time.time()
for a in [1..1000]: #default precision must be > max a.
    c=p2RecipPowerConstantTerm(a)
    o=order(3,c)
    ds=digitsum(3,a)
    finish=time.time()
    test=mod(c/3^o,3)
    tests=tests+[test]
    print((a,test,finish-start))
print(tests)

(1, 1, 0.019771814346313477)
(2, 1, 0.03896307945251465)
(3, 1, 0.04501986503601074)
(4, 1, 0.052870988845825195)
(5, 1, 0.06117606163024902)
(6, 1, 0.0739130973815918)
(7, 1, 0.0908968448638916)
(8, 1, 0.10569596290588379)
(9, 1, 0.12229609489440918)
(10, 1, 0.1380600929260254)
(11, 1, 0.15320920944213867)
(12, 1, 0.16803193092346191)
(13, 1, 0.18289685249328613)
(14, 1, 0.1977400779724121)
(15, 1, 0.21264410018920898)
(16, 1, 0.2276301383972168)
(17, 1, 0.24972796440124512)
(18, 1, 0.2640960216522217)
(19, 1, 0.2785179615020752)
(20, 1, 0.294205904006958)
(21, 1, 0.31261491775512695)
(22, 1, 0.328279972076416)
(23, 1, 0.3439209461212158)
(24, 1, 0.35930395126342773)
(25, 1, 0.3745579719543457)
(26, 1, 0.3899240493774414)
(27, 1, 0.4053151607513428)
(28, 1, 0.42082905769348145)
(29, 1, 0.43613696098327637)
(30, 1, 0.45127391815185547)
(31, 1, 0.466616153717041)
(32, 1, 0.48181605339050293)
(33, 1, 0.4972670078277588)
(34, 1, 0.5125241279602051)
(35, 1, 0.5276570320129395)
(36, 1, 0.54

(294, 1, 9.285552978515625)
(295, 1, 9.349916934967041)
(296, 1, 9.409950017929077)
(297, 1, 9.473960876464844)
(298, 1, 9.538756847381592)
(299, 1, 9.606979131698608)
(300, 1, 9.673296928405762)
(301, 1, 9.74189281463623)
(302, 1, 9.809988975524902)
(303, 1, 9.879633903503418)
(304, 1, 9.9425048828125)
(305, 1, 10.006776094436646)
(306, 1, 10.071754932403564)
(307, 1, 10.141286849975586)
(308, 1, 10.207106113433838)
(309, 1, 10.27579116821289)
(310, 1, 10.344465970993042)
(311, 1, 10.416049003601074)
(312, 1, 10.483147144317627)
(313, 1, 10.551867008209229)
(314, 1, 10.620473861694336)
(315, 1, 10.693865060806274)
(316, 1, 10.763617992401123)
(317, 1, 10.836225986480713)
(318, 1, 10.909103155136108)
(319, 1, 10.987823247909546)
(320, 1, 11.04875111579895)
(321, 1, 11.113704919815063)
(322, 1, 11.178008079528809)
(323, 1, 11.24533987045288)
(324, 1, 11.312036037445068)
(325, 1, 11.38039493560791)
(326, 1, 11.448920011520386)
(327, 1, 11.519652843475342)
(328, 1, 11.586652994155884)
(32

(583, 1, 50.09427499771118)
(584, 1, 50.316920042037964)
(585, 1, 50.55835795402527)
(586, 1, 50.80257201194763)
(587, 1, 51.06699180603027)
(588, 1, 51.31108617782593)
(589, 1, 51.574506998062134)
(590, 1, 51.83883500099182)
(591, 1, 52.1437771320343)
(592, 1, 52.40809202194214)
(593, 1, 52.694206953048706)
(594, 1, 52.99374604225159)
(595, 1, 53.303297996520996)
(596, 1, 53.58515691757202)
(597, 1, 53.884921073913574)
(598, 1, 54.19451403617859)
(599, 1, 54.52719807624817)
(600, 1, 54.82463598251343)
(601, 1, 55.11781191825867)
(602, 1, 55.4666051864624)
(603, 1, 55.773488998413086)
(604, 1, 56.05357003211975)
(605, 1, 56.3582079410553)
(606, 1, 56.65695381164551)
(607, 1, 56.98407602310181)
(608, 1, 57.2271192073822)
(609, 1, 57.48894214630127)
(610, 1, 57.74554800987244)
(611, 1, 58.02100110054016)
(612, 1, 58.272237062454224)
(613, 1, 58.54295802116394)
(614, 1, 58.815911054611206)
(615, 1, 59.1105420589447)
(616, 1, 59.36770987510681)
(617, 1, 59.64034295082092)
(618, 1, 59.91400

(873, 1, 162.04203391075134)
(874, 1, 162.56927013397217)
(875, 1, 163.15000987052917)
(876, 1, 163.6750509738922)
(877, 1, 164.24253106117249)
(878, 1, 164.81218695640564)
(879, 1, 165.43672800064087)
(880, 1, 165.94221997261047)
(881, 1, 166.48943424224854)
(882, 1, 167.04243683815002)
(883, 1, 167.63934588432312)
(884, 1, 168.1995530128479)
(885, 1, 168.79710698127747)
(886, 1, 169.397625207901)
(887, 1, 170.01428198814392)
(888, 1, 170.57232093811035)
(889, 1, 171.15556621551514)
(890, 1, 171.7396538257599)
(891, 1, 172.35714101791382)
(892, 1, 172.9373791217804)
(893, 1, 173.5551381111145)
(894, 1, 174.17574310302734)
(895, 1, 174.8446180820465)
(896, 1, 175.29557085037231)
(897, 1, 175.78070998191833)
(898, 1, 176.26635813713074)
(899, 1, 176.7908239364624)
(900, 1, 177.27780604362488)
(901, 1, 177.81433081626892)
(902, 1, 178.3437750339508)
(903, 1, 178.90411400794983)
(904, 1, 179.4006769657135)
(905, 1, 179.93261098861694)
(906, 1, 180.45060801506042)
(907, 1, 181.011137008667